<a href="https://colab.research.google.com/github/Ksen1aT/your-job-offer-1/blob/parser/Match_user_and_vacancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=cbfa0dadbdf07007d9afabea3c85f1d3b113a80d53f508acd29cb2b38e6afea7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 14.0 MB/s eta 0:00:00


In [4]:
!pip install mpld3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 3.9 MB/s eta 0:00:00


In [5]:
!pip install sqlalchemy

In [6]:
import sqlalchemy
import requests
from datetime import datetime, timedelta
import pandas as pd
import time
import random
import json
import time
import os
import json
import numpy as np
import re
from requests.exceptions import HTTPError, Timeout, ConnectionError
import rdflib
import nltk
from nltk.tokenize import sent_tokenize
from string import punctuation
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk import download as nltk_download
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import pymorphy2
from pymorphy2 import MorphAnalyzer

from sklearn import feature_extraction
import mpld3
from collections import Counter
import torch
from dataclasses import dataclass

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
@dataclass
class Vacancy:
    """Класс для хранения информации о вакансиях."""
    vacancy_id: int
    job_title: str
    response_letter_required: bool = False
    country: str = ""
    city: str = ""
    district: str = ""
    salary: list = None
    office_address: str = ""
    subway_station: str = ""
    employer_information: str = ""
    requirements: str = ""
    work_schedule_working_days: list = None
    work_schedule_time_intervals: list = None
    experience: str = ""
    remote_work: bool = False

In [ ]:
@dataclass
class Resume:
    resume_id: int
    user_id: int
    title: str = ""
    job_title: str = ""
    country: str = ""
    city: str = ""
    district: str = ""
    salary: int = 0
    work_schedule_working_days: list = None
    work_schedule_time_intervals: list = None
    experience: int = 0
    remote_work: bool = False
    education: str = ""
    additional_information: str = ""

In [ ]:
def clean(word):
    return re.sub(r"[^A-ZА-Яa-zа-я+\s]", "", word)

In [ ]:
stop_words = stopwords.words('russian') + ["опыт",
                                           "навык",
                                           "знание",
                                           "понимание",
                                           "работа",
                                           "умение",
                                           "большой",
                                           "хороший",
                                           "прикладной",
                                           "некоторый",
                                           "небольшой"]


In [ ]:
def tokenize_text(text):
    if text == None:
      return
    text = text.lower()
    text = text.translate(
        str.maketrans(punctuation, " " * len(punctuation))
    )
    word_tokens = word_tokenize(text)
    word_tokens = [clean(word) for word in word_tokens]
    morph = MorphAnalyzer()
    lemmatize_words = [morph.normal_forms(word)[0] for word in word_tokens]
    filtered_tokens = [word for word in lemmatize_words if word not in stop_words]
    return filtered_tokens

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

def get_word_embeddings(words):
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states[:, 0, :].cpu().numpy()
    return embeddings

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return float(dot_product / (norm_a * norm_b))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
def check_vacancy(requirements, skills):
  tok_req = tokenize_text(requirements)
  tok_skills = tokenize_text(skills)
  if tok_req == None or tok_skills == None:
    return False
  for i in tok_req:
    for j in tok_skills:
      if i == j:
        return True
  return False

In [ ]:
def match_skills_and_requirements(requirements, skills):
  if requirements == None or skills == None:
    return 0
  emb_requirements = get_word_embeddings(requirements)
  emb_skills = get_word_embeddings(skills)
  similarity = 0
  max_similarity = 0
  for i in emb_requirements:
    for j in emb_skills:
      similarity += cosine_similarity(i, j)
      max_similarity += 1
  return similarity / max_similarity

In [ ]:
 #Россия, Белоруссия, Казахстан, Армения, Кыргызстан, Молдавия, Таджикистан

In [ ]:
currency = {'RUB': 1,
            'BYN': 31.05,
            'KZT': 0.19,
            'AMD': 0.25,
            'KGS': 1.16,
            'MDL': 5.5,
            'TJS': 9.22,
            'UZS': 0.0078}

In [ ]:
def match_vacancies(vacancies, resume):
  similarity_with_vacancies = {}
  idx = 0
  for vacancy in vacancies:
    similarity = 0
    idx += 1
    print(idx)
    if resume.country != "" and vacancy.country != "" and vacancy.country != resume.country:
      continue
    if resume.city != "" and vacancy.city != "" and vacancy.city != resume.city:
      continue
    if vacancy.remote_work == False and resume.remote_work == True:
      continue
    if check_vacancy(vacancy.requirements, resume.additional_information) == False:
      continue
    if vacancy.salary[0] != None and vacancy.salary[2] in currency:
       if vacancy.salary[1] != None and resume.salary * 0.9 <= vacancy.salary[1] * currency[vacancy.salary[2]]:
          similarity += 0.1
       elif vacancy.salary[1] == None and resume.salary * 1.1 >= vacancy.salary[0] * currency[vacancy.salary[2]]:
          similarity += 0.1
    experience_min = 0
    if vacancy.experience == 'От 1 года до 3 лет':
      experience_min = 1
    elif vacancy.experience == 'От 3 до 6 лет':
      experience_min = 3
    elif vacancy.experience == 'Более 6 лет':
      experience_min = 6
    if resume.experience >= experience_min:
      similarity += 0.1
    similarity += match_skills_and_requirements(vacancy.requirements, resume.additional_information)
    similarity += match_skills_and_requirements(vacancy.job_title, resume.job_title)
    if similarity in similarity_with_vacancies:
      similarity_with_vacancies[similarity].append(vacancy)
    else:
      similarity_with_vacancies[similarity] = [vacancy]
  return similarity_with_vacancies

In [ ]:
resume = Resume(resume_id = 1,
                user_id = 1,
                additional_information = "автомобиль")

In [ ]:
print(hh_vacancies[:10])

[Vacancy(vacancy_id=112489227, job_title='SMM-специалист', response_letter_required=False, country='Беларусь', city='Минск', district='улица Чеботарёва', salary=[None, None, None], office_address='Минск, улица Чеботарёва, 7А', subway_station='Тракторный завод', employer_information='Макси Бай Медиа', requirements='Есть опыт работы с клиентами или в интернет-агентстве? - Умеете презентовать, доказывать свою точку зрения? - Готовы к динамичной работе в...', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='От 1 года до 3 лет', remote_work=False), Vacancy(vacancy_id=112482071, job_title='Водитель персональный', response_letter_required=False, country='Казахстан', city='Астана', district='улица Акмола', salary=[500000, 800000, 'KZT'], office_address='Астана, улица Акмола', subway_station='', employer_information='Андеграунд', requirements='1. Строгое соблюдение стандартов сервиса и дресс-кода (костюм, галстук, туфли). 3. Пунктуальность — обязательное требовани

In [ ]:
sim = match_vacancies(hh_vacancies[:200], resume)
sim = sorted(sim.items(), reverse = True)
num = 10
print(sim[:num])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
[(0.7769478559494019, [Vacancy(vacancy_id=112627427, job_title='Личный водитель', response_letter_required=False, country='Россия', city='Москва', district='Пресненская набережная', salary=[120000, None, 'RUR'], office_address='Москва, Пресненская набережная, 10с2', subway_station='', employer_information='Такс Комплаенс', requirements='Обязательное наличие прав категории В. Знание ПДД, административного законодательства в области ДД. Знание устройства автомобиля, его технических характеристик. ', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='Более 6 лет', remote_work=False)]), (0.7516686916351318, [Vacancy(vacancy_id=112608260, job_title='Водитель-экспедитор', response_letter_required=False, country='Россия', city='Нижний Новгород', district='Московское шоссе', salary=[80000, 100000, 'RUR'], office_address='Нижний Новгород, Московское шоссе, 302/1', subway_station='', employer_information='Планета Шин', requirements='В

In [ ]:
resume = Resume(resume_id = 1,
                user_id = 1,
                additional_information = "большой опыт в продажах")
sim = match_vacancies(hh_vacancies[:200], resume)
sim = sorted(sim.items(), reverse = True)
num = 10
print(sim[:num])

1
2
3
4
[(0.8875036835670471, [Vacancy(vacancy_id=112577058, job_title='Менеджер группы прямых продаж (DSA)', response_letter_required=False, country='Россия', city='', district='', salary=[60000, None, 'RUR'], office_address='', subway_station='', employer_information='Банк ВТБ (ПАО)', requirements='Среднее/среднее профессиональное, незаконченное/высшее образование. Опыт работы в продажах будет являться преимуществом. Желание развиваться в банковской сфере. ', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='Нет опыта', remote_work=False)]), (0.8583747744560242, [Vacancy(vacancy_id=112622889, job_title='Менеджер по обучению и развитию', response_letter_required=False, country='Россия', city='Москва', district=None, salary=[280000, None, 'RUR'], office_address='Москва', subway_station='Таганская', employer_information='Alta Personnel', requirements='Высшее профессиональное образование (медицинское, фармацевтическое, психологическое или любое другое). Опыт

In [ ]:
resume = Resume(resume_id = 1,
                user_id = 1,
                additional_information = "автомобили")
sim = match_vacancies(hh_vacancies[:30], resume)
sim = sorted(sim.items(), reverse = True)
num = 10
print(sim[:num])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
[(1.721099591255188, [Vacancy(vacancy_id=112593600, job_title='Водитель в семью', response_letter_required=False, country='Казахстан', city='', district='', salary=[300000, 400000, 'KZT'], office_address='', subway_station='', employer_information='Aipack', requirements='Имеющего опыт работы в семье. Аккуратное вождение. Порядочность. Пунктуальность. Наличие собственного автомобиля.', work_schedule_working_days=None, work_schedule_time_intervals=None, experience='Более 6 лет', remote_work=False)]), (1.7121224761009217, [Vacancy(vacancy_id=112595789, job_title='Водитель', response_letter_required=False, country='Казахстан', city='Астана', district='проспект Мангилик Ел', salary=[300000, 350000, 'KZT'], office_address='Астана, проспект Мангилик Ел, 20', subway_station='', employer_information='Мерей-Инжиниринг LTD', requirements='Личный автомобиль.', work_schedule_working_days=None, work_schedule_time_interv